In [1]:

!pip install optuna_integration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 18.3 MB/s eta 0:00:00


In [2]:
import os
import random
from tqdm import tqdm
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

import numpy as np
import pandas as pd

from sklearn.metrics import make_scorer, mean_squared_error, mean_gamma_deviance
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import VotingRegressor
from sklearn.impute import SimpleImputer, KNNImputer

from scipy.optimize import minimize
import optuna

import optuna.integration.lightgbm as lgb

#from sklearn.ensemble import RandomForestRegressor
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer

import warnings
warnings.filterwarnings('ignore')


SEED = 42

In [5]:
root = Path('/content')
df_train = pd.read_csv(root / 'sample_loss_data_shape.csv', parse_dates=["yyyymm"])
#df_test = pd.read_csv(root / 'test.csv')

In [6]:
df_train['diff'] = df_train['input_qty'] - df_train['output_qty']
df_train["month"] = pd.to_datetime(df_train["yyyymm"]).dt.to_period("M")
df_train.head()

,yyyymm,product_id,process_id,input_qty,output_qty,shape_mm,diff,month
0,2023-01-01,P-001,ProcA,9802,9367,4.2,435,2023-01
1,2023-01-01,P-001,ProcB,8909,8639,4.2,270,2023-01
2,2023-01-01,P-001,ProcC,11434,10912,4.2,522,2023-01
3,2023-01-01,P-002,ProcA,11311,10854,5.0,457,2023-01
4,2023-01-01,P-002,ProcB,10802,10515,5.0,287,2023-01


In [7]:
feature_cols = ['product_id', 'process_id', 'input_qty', 'shape_mm']
target_col = 'diff'

In [8]:
df_train = df_train.dropna(subset=[target_col])
df_train[target_col].isnull().any()

np.False_

In [9]:
# 初期訓練期間
start_test = df_train["month"].min() + 12  # 12 ヶ月後から予測開始
months = df_train["month"].unique()
results = []

train_idx = df_train["month"] < start_test

In [10]:
df_train_hy = df_train[train_idx]
df_train_hy["diff"] = df_train_hy["diff"].astype("float64")
df_train_hy.tail()

,yyyymm,product_id,process_id,input_qty,output_qty,shape_mm,diff,month
535,2023-12-01,P-004,ProcE,10285,9988,6.1,297.0,2023-12
536,2023-12-01,P-004,ProcG,8028,7637,6.1,391.0,2023-12
537,2023-12-01,P-005,ProcD,9494,9152,4.9,342.0,2023-12
538,2023-12-01,P-005,ProcE,8373,7992,4.9,381.0,2023-12
539,2023-12-01,P-005,ProcG,11334,10940,4.9,394.0,2023-12


In [11]:
df_train_hy[df_train_hy['diff']>0]

,yyyymm,product_id,process_id,input_qty,output_qty,shape_mm,diff,month
0,2023-01-01,P-001,ProcA,9802,9367,4.2,435.0,2023-01
1,2023-01-01,P-001,ProcB,8909,8639,4.2,270.0,2023-01
2,2023-01-01,P-001,ProcC,11434,10912,4.2,522.0,2023-01
3,2023-01-01,P-002,ProcA,11311,10854,5.0,457.0,2023-01
4,2023-01-01,P-002,ProcB,10802,10515,5.0,287.0,2023-01
...,...,...,...,...,...,...,...,...
535,2023-12-01,P-004,ProcE,10285,9988,6.1,297.0,2023-12
536,2023-12-01,P-004,ProcG,8028,7637,6.1,391.0,2023-12
537,2023-12-01,P-005,ProcD,9494,9152,4.9,342.0,2023-12
538,2023-12-01,P-005,ProcE,8373,7992,4.9,381.0,2023-12


In [56]:
gamma_scorer = make_scorer(mean_gamma_deviance)
rmse_scorer2 = make_scorer(mean_squared_error, squared=False)
rmse_scorer = make_scorer(mean_squared_error)

In [62]:
!pip install -U scikit-learn

In [64]:
#python - <<'PY'
import sklearn, pkg_resources, sys, platform
print("sklearn  : ", sklearn.__version__)
print("dist-info: ", pkg_resources.get_distribution("scikit-learn").version)
#PY

sklearn  :  1.6.1
dist-info:  1.6.1


In [75]:
class CustomLGBMRegressor2(lgb.LGBMRegressor):
    def __init__(self, random_state, **kwargs):
        if 'random_state' in kwargs:
            random_state = kwargs.pop('random_state')
        super().__init__(random_state=random_state, **kwargs)
        self.enc_prod = OrdinalEncoder(
                          #categories=categories,
                          dtype=np.int32,
                          handle_unknown='use_encoded_value',
                          unknown_value=-1,
                          encoded_missing_value=-2,
                          min_frequency=2)
        self.enc_proc = OrdinalEncoder(
                          #categories=categories,
                          dtype=np.int32,
                          handle_unknown='use_encoded_value',
                          unknown_value=-1,
                          encoded_missing_value=-2,
                          min_frequency=2)

        #self.imputer = IterativeImputer(RandomForestRegressor())


#    def _encode(self, X, fit=False):
#        X = X.copy()
#        for col in ['product_id', 'process_id']:
#            X[col] = X[col].astype('category')
#        return X

    def _encode(self, X, fit=False):
        X = X.copy()
        if fit:
            X['product_id'] = self.enc_prod.fit_transform(X[['product_id']]).ravel()
            X['process_id'] = self.enc_proc.fit_transform(X[['process_id']]).ravel()
        else:
            X['product_id'] = self.enc_prod.transform(X[['product_id']]).ravel()
            X['process_id'] = self.enc_proc.transform(X[['process_id']]).ravel()
        return X

    def fit(self, X, y, **kwargs):
        X_enc = self._encode(X, fit=True)
        super().fit(X_enc, y, **kwargs)


#        super().fit(X, y, **kwargs)
        y_pred = super().predict(X_enc, **kwargs)
        #return self
#        X_enc = self._encode(X, fit=True)
#        super().fit(X, y, categorical_feature=['product_id','process_id'], **kwargs)
        print(f'X:{X_enc.head()}')
        print(f'y:{y}')
        print(f'y_pred-y:{y_pred-y}')
        print(self.random_state)
        return self

    def predict(self, X, **kwargs):
        X_enc = self._encode(X, fit=False)
        y_pred = super().predict(X_enc, **kwargs)
        return y_pred

In [ ]:
#Nested CV
# ---------------- データ ---------------------
X = df_train_hy[feature_cols]
y = df_train_hy[target_col]

outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
outer_scores = []
best_params_each_fold = []

for outer_i, (train_idx, test_idx) in enumerate(outer_cv.split(X, y), 1):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    inner_cv = KFold(n_splits=3, shuffle=True, random_state=outer_i)

    # -------- Optuna objective ---------------
    def lgb_objective(trial):
      params = {
          'objective':         'gamma',
          'verbosity':         -1,
           #'n_iter':            200,
          'random_state':      SEED,
          'boosting_type':     'gbdt',
          #"lambda_l1":        trial.suggest_float("lambda_l1", 1e-3, 0.01, log=True),
          #'lambda_l2':         trial.suggest_float('lambda_l2', 1e-3, 0.01, log=True),
          'learning_rate':     trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
          #'max_depth':         trial.suggest_int('max_depth', 2, 2),
          #'num_leaves':        trial.suggest_int('num_leaves', 2, 2),
          #'colsample_bytree':  trial.suggest_float('colsample_bytree', 0.4, 1.0),
          #'colsample_bynode':  trial.suggest_float('colsample_bynode', 0.4, 1.0),
          #'bagging_fraction':  trial.suggest_float('bagging_fraction', 0.4, 1.0),
          #'bagging_freq':      trial.suggest_int('bagging_freq', 1, 2),
          'min_data_in_leaf':  trial.suggest_int('min_data_in_leaf', 1, 2),
      }

      cv = KFold(3, shuffle=True, random_state=SEED)
      estimator = CustomLGBMRegressor2(**params)
      #print(f'X:{X.head()}')
      #print(f'y:{y}')

      val_scores = cross_val_score(
            estimator=estimator,
            X=X_train, y=y_train,
            cv=inner_cv,
            scoring=gamma_scorer,
        )


      return np.mean(val_scores)

    study = optuna.create_study(direction='minimize', study_name='Regressor')
    study.optimize(lgb_objective, n_trials=10, show_progress_bar=True)

    best_params = study.best_trial.params
    best_params_each_fold.append(best_params)

    # -------- Outer-test 評価 ----------------
    final_model = CustomLGBMRegressor2(**params)

    # early-stopping を効かせる
    final_model.fit(
        X_train, y_train,
        #eval_set=[(X_test, y_test)],
        #early_stopping_rounds=50,
        #verbose=False
    )

    y_pred = final_model.predict(X_test)
    gamma = mean_gamma_deviance(y_test, y_pred)
    outer_scores.append(gamma)
    print(f"Fold {outer_i} gamma = {gamma:.4f}")

# ------------- まとめ ------------------------
print("\n=== Nested CV result ===")
print(f"Mean gamma : {np.mean(outer_scores):.4f} ± {np.std(outer_scores):.4f}")

pd.set_option("display.max_columns", None)
print(pd.DataFrame(best_params_each_fold))

In [ ]:
def lgb_objective(trial):
    params = {
        'objective':         'gamma',
        'verbosity':         -1,
        #'n_iter':            200,
        'random_state':      SEED,
        'boosting_type':     'gbdt',
        #"lambda_l1":        trial.suggest_float("lambda_l1", 1e-3, 0.01, log=True),
        #'lambda_l2':         trial.suggest_float('lambda_l2', 1e-3, 0.01, log=True),
        'learning_rate':     trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
        #'max_depth':         trial.suggest_int('max_depth', 2, 2),
        #'num_leaves':        trial.suggest_int('num_leaves', 2, 2),
        #'colsample_bytree':  trial.suggest_float('colsample_bytree', 0.4, 1.0),
        #'colsample_bynode':  trial.suggest_float('colsample_bynode', 0.4, 1.0),
        #'bagging_fraction':  trial.suggest_float('bagging_fraction', 0.4, 1.0),
        #'bagging_freq':      trial.suggest_int('bagging_freq', 1, 2),
        'min_data_in_leaf':  trial.suggest_int('min_data_in_leaf', 1, 2),
    }

    X = df_train_hy[feature_cols]
    y = df_train_hy[target_col]
    #cv = StratifiedKFold(5, shuffle=True, random_state=SEED)
    cv = KFold(3, shuffle=True, random_state=SEED)
    estimator = CustomLGBMRegressor2(**params)
    #print(f'X:{X.head()}')
    #print(f'y:{y}')

    val_scores = cross_val_score(
        estimator=estimator,
        X=X, y=y,
        cv=cv,
        scoring=gamma_scorer,
    )

    return np.mean(val_scores)

In [ ]:
params = study.best_params

model = CustomLGBMRegressor2(**params, random_state=SEED)


In [81]:
import shap

In [ ]:

# 初期訓練期間
start_test = df_train["month"].min() + 12  # 12 ヶ月後から予測開始
months = df_train["month"].unique()
results = []

train_idx = df_train["month"] < start_test

for m in tqdm(months[months >= start_test]):
    # --- 学習データ / 予測データ ---
    X_train, y_train = df_train.loc[train_idx, feature_cols], df_train.loc[train_idx, target_col]
    X_pred  = df_train.loc[df_train["month"] == m, feature_cols]
    y_true  = df_train.loc[df_train["month"] == m, target_col]
    #w_pred  = df_train.loc[df_train["month"] == m, "inp"]          # 重み用

    # --- モデル再学習（必要なら 3 ヶ月ごと check） ---
    model.fit(X_train, y_train)

    # --- 予測 & 残差 ---
    y_hat = model.predict(X_pred)
    df_train.loc[df_train["month"] == m, "y_hat"] = y_hat
    df_train.loc[df_train["month"] == m, "residual"] = y_true - y_hat

    # --- KPI 集計（工程 A を例） ---
    #mask_kpi = (df["month"] == m) & (df["process_id"] == "A")
    #kpi_exp  = (df.loc[mask_kpi, "y_hat"] * w_pred[mask_kpi]).sum() / w_pred[mask_kpi].sum()
    #kpi_act  = (df.loc[mask_kpi, "loss_rate"] * w_pred[mask_kpi]).sum() / w_pred[mask_kpi].sum()
    #delta_kpi = kpi_act - kpi_exp

    # --- Δ分解 ---
    #sub = df.loc[mask_kpi].copy()
    #sub["delta_qty"] = (sub["loss_rate"] - sub["y_hat"]) * sub["inp"]
    #contrib = (sub.groupby("product_id")["delta_qty"].sum()
    #             .sort_values(key=abs, ascending=False).head(10))

    # --- SHAP（任意） ---
    expl = shap.TreeExplainer(model)
    shap_vals = expl.shap_values(X_pred, check_additivity=False)
    df_train.loc[df_train["month"] == m, "shap"] = shap_vals
    #shap_df   = pd.DataFrame(shap_vals, columns=X_pred.columns).mul(w_pred.values, axis=0)
    #shap_prod = shap_df.filter(like="product_").sum().sort_values(key=abs, ascending=False).head(10)

    #results.append({
    #    "month": m, "kpi_exp": kpi_exp, "kpi_act": kpi_act,
    #    "delta": delta_kpi, "top_delta": contrib, "top_shap": shap_prod
    #})

    # --- 訓練セットに当月を追加して次ループへ ---
    train_idx |= (df_train["month"] == m)